Задание 2. Вам предоставлены данные о подаче заявок для участия в закупочных конкурсах.
1.	Провести анализ случаев подачи заявок разными поставщиками с одного адреса.
2.	Наглядно визуализировать результат анализа в виде отчета в Qlik Sense. Предусмотреть возможность фильтрации результатов по диапазону количества совместных подач (напр., 10-20, 20-50 и т.д.) и разницы во времени между подачей первой и второй заявки (напр., 0-10 минут, 10-30 и т.д.)

In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

pd.options.display.max_columns = 20
pd.options.display.max_rows = 10

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Наблюдаются лишние запятые в исходном файле, исправим это.

In [7]:
my_list =[]
with open('C:\\Users\\Mell\\Desktop\\Task_2\\Task 2 - data.csv', 'r') as r, open('C:\\Users\\Mell\\Desktop\\Task_2\\Task 2_data_red.csv', 'w') as w:    
    for num, line in enumerate(r):
        if num > 0:
            my_list = line.split('Company')
            newline = my_list[0].replace(',','') + ',Company' + my_list[1]
        else:
            newline = line
        my_list = []
        w.write(newline)

In [8]:
data = pd.read_csv('C:\\Users\\Mell\\Desktop\\Task_2\\Task 2_data_red.csv', sep = ',', engine='python')

In [9]:
data.head()

,Tender_ID,Поставщик_1,Received_DateTime,IP_address
0,Tender 1,Company 107,1-15-16 18:01,IP address 14598
1,Tender 1,Company 107,1-15-16 18:01,IP address 14598
2,Tender 1,Company 107,1-15-16 18:01,IP address 14598
3,Tender 1,Company 1185,1-15-16 17:26,IP address 3620
4,Tender 1,Company 182,1-15-16 17:25,IP address 2951


Посчитаем количество поставщиков с одинаковым IP

In [16]:
data['Количество_компаний'] = data.groupby('IP_address')['Поставщик_1'].transform(pd.Series.nunique)

In [17]:
data

,Tender_ID,Поставщик_1,Received_DateTime,IP_address,Количество_компаний
0,Tender 1,Company 107,1-15-16 18:01,IP address 14598,1
1,Tender 1,Company 107,1-15-16 18:01,IP address 14598,1
2,Tender 1,Company 107,1-15-16 18:01,IP address 14598,1
3,Tender 1,Company 1185,1-15-16 17:26,IP address 3620,1
4,Tender 1,Company 182,1-15-16 17:25,IP address 2951,9
...,...,...,...,...,...
74560,Tender 9999,Company 317,1-27-17 17:58,IP address 17475,1
74561,Tender 7746,Company 100,3-16-17 14:05,IP address 5476,2
74562,Tender 7746,Company 202,3-16-17 17:28,IP address 10272,1
74563,Tender 7746,Company 427,3-16-17 17:20,IP address 769,1


Теперь добавим в новый столбец список компаний, которые подавали заявки с этого адреса.

In [66]:
name_comp = data.copy()

In [67]:
name_comp = name_comp.drop(columns =['Tender_ID', 'Received_DateTime', 'Количество_компаний'])

In [69]:
name_comp = name_comp.groupby('IP_address')['Поставщик_1'].unique().to_frame(name="Компании")

In [71]:
name_comp.head(3)

,Компании
IP_address,
IP address 1,[Company 1152]
IP address 10,[Company 1152]
IP address 100,[Company 915]


In [74]:
data = data.merge(name_comp, on ='IP_address')

In [78]:
data.head(3)

,Tender_ID,Поставщик_1,Received_DateTime,IP_address,Количество_компаний,Компании
0,Tender 1,Company 107,1-15-16 18:01,IP address 14598,1,[Company 107]
1,Tender 1,Company 107,1-15-16 18:01,IP address 14598,1,[Company 107]
2,Tender 1,Company 107,1-15-16 18:01,IP address 14598,1,[Company 107]


In [181]:
data['Received_DateTime'] = pd.to_datetime(data['Received_DateTime'])

Оставим только те IP, с которых подавали заявки несколько компаний. 

In [186]:
data_ = data.loc[data['Количество_компаний'] > 1]

Вычислим время между подачей первой и второй заявками.

In [187]:
data_['Время_между_заявками'] = np.nan

In [188]:
for i in data_['IP_address'].unique():
    data_['Время_между_заявками'].loc[data_['IP_address'] == i] = data_.loc[data_['IP_address'] == i].sort_values('Received_DateTime').iloc[1]['Received_DateTime'] - data_.loc[data_['IP_address'] == i].sort_values('Received_DateTime').iloc[0]['Received_DateTime']

Приведем время к минутам, по условию задания.

In [236]:
data_['Время_между_заявками'] = pd.to_timedelta(data_['Время_между_заявками'])
data_['Время между заявками (мин)'] = data_['Время_между_заявками'].dt.total_seconds().div(60).astype(int)

In [323]:
data_to_save = data_.copy()

In [324]:
data_to_save = data_to_save.drop(columns =['Tender_ID', 'Received_DateTime', 'Поставщик_1','Время_между_заявками'])

Посчитаем количество совместных заявок. 

In [325]:
data_to_save['Количество_заявок'] = data_to_save.groupby('IP_address').IP_address.transform(np.size)

In [326]:
data_to_save = data_to_save.drop_duplicates('IP_address', keep='first')

In [327]:
data_to_save['Компании'] = data_to_save['Компании'].apply(', '.join)

In [328]:
data_to_save.columns = ['IP_address','Number of companies', 'Companies', 'Time between requests', 'Number of requests']

In [329]:
data_to_save

,IP_address,Number of companies,Companies,Time between requests,Number of requests
9,IP address 2951,9,"Company 182, Company 122, Company 616, Company...",15860,68
95,IP address 10814,2,"Company 77, Company 81",21,2
98,IP address 16751,2,"Company 1764, Company 926",52,2
103,IP address 11971,3,"Company 533, Company 531, Company 532",7,44
153,IP address 15331,2,"Company 124, Company 184",2,12
...,...,...,...,...,...
74535,IP address 11011,2,"Company 236, Company 237",5,2
74537,IP address 11647,2,"Company 1337, Company 308",12,2
74540,IP address 8817,2,"Company 92, Company 96",25,3
74556,IP address 16575,2,"Company 1000, Company 1175",156,5


**В результате работы получена таблица содержащая:  
    1.IP,  
    2.Количество компаний, которые подали заявку с этого адреса (>1),  
    3.Список компаний, которые подали заявку с этого адреса (>1),   
    4.Время между первой и второй заявкой,  
    5.Совместное количество подач заявок.**  

In [331]:
data_to_save.to_csv('C:\\Users\\Mell\\Desktop\\Task_2\\data_to_save.csv', sep=';', index=False)